In [213]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo

In [214]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [215]:
db = client.recipe_db
collection = db.low_calorie

In [216]:
url = 'https://www.allrecipes.com/recipes/84/healthy-recipes/'

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [217]:
results = soup.find_all('a', class_='carouselNav__link recipeCarousel__link')
results

[<a class="carouselNav__link recipeCarousel__link" data-tracking-content-headline="healthy appetizers" href="https://www.allrecipes.com/recipes/1319/healthy-recipes/appetizers/" tabindex="0">
 <div alt="Healthy Appetizers" aria-label="" class="component lazy-image lazy-image-udf aspect_1x1 align-default" data-alt="Healthy Appetizers" data-content-type="" data-crop-percentage="100" data-high-density="false" data-main-recipe="" data-orientation="default" data-original-height="1277" data-original-width="1278" data-shop-image="false" data-special-crop="" data-src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F07%2F22%2F6583475.jpg" data-title="Healthy Appetizers" data-tracking-zone="image" data-width="      100
                                    ">
 <noscript>
 <img alt="Healthy Appetizers" src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2

In [218]:
health_cats = ['low-calorie recipes','low-carb recipes','low-cholesterol recipes','low-fat recipes','weight-loss recipes','healthy snacks']

In [219]:
cat =  [x['href'] for x in results if x['data-tracking-content-headline'] in health_cats]
cat

['https://www.allrecipes.com/recipes/12155/healthy-recipes/snacks/',
 'https://www.allrecipes.com/recipes/1232/healthy-recipes/low-calorie/',
 'https://www.allrecipes.com/recipes/742/healthy-recipes/low-carb/',
 'https://www.allrecipes.com/recipes/737/healthy-recipes/low-cholesterol/',
 'https://www.allrecipes.com/recipes/1231/healthy-recipes/low-fat/',
 'https://www.allrecipes.com/recipes/22607/healthy-recipes/weight-loss/']

In [220]:
main_cats={}

for c in cat:
    r = requests.get(c)
    soup = bs(r.text, 'html.parser')
    title = soup.find('h1', class_= 'category-page-heading').text
    results = soup.find_all('a', class_= 'card__titleLink manual-link-behavior')
    links = [y['href'] for y in results]
    
    main_cats[title] = list(set(links))
    
main_cats

{'Healthy Snack Recipes': ['https://www.allrecipes.com/recipe/223467/granola-bars/',
  'https://www.allrecipes.com/recipe/15006/spiced-sweet-roasted-red-pepper-hummus/',
  'https://www.allrecipes.com/recipe/219456/healthier-buffalo-chicken-dip/',
  'https://www.allrecipes.com/recipe/19677/great-garlic-knots/',
  'https://www.allrecipes.com/recipe/236557/salt-and-vinegar-potato-chips/',
  'https://www.allrecipes.com/recipe/20482/baked-tortilla-chips/',
  'https://www.allrecipes.com/recipe/24720/texas-caviar-i/',
  'https://www.allrecipes.com/recipe/54165/balsamic-bruschetta/',
  'https://www.allrecipes.com/recipe/214722/pickled-daikon-radish-and-carrot/',
  'https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/',
  'https://www.allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/',
  'https://www.allrecipes.com/recipe/236025/salt-and-pepper-skillet-fries/',
  'https://www.allrecipes.com/recipe/239017/fresh-tomato-salsa-restaurant-style/',
  'https://www.allrecipes.

In [221]:
for k, v in main_cats.items():
    collection = db[k]
    for u in v:
        print(u)
        resp = requests.get(u)
        soup = bs(resp.text, 'html.parser')
        title = soup.find('h1', class_= 'headline heading-content').text
        
        try:
            rating = soup.find('span', class_='review-star-text').text.replace('\n',"").strip()
        except:
            rating = 'none'
        
        try:
            num_ratings = soup.find('span', class_='ugc-ratings-item').text.replace('\n',"").strip()
        except:
            num_ratings = 'none'
            
        
        time = soup.find_all('div', class_= 'recipe-meta-item')
        time_dict = {}
        time_dict['prep'] = time[0].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
        time_dict['total'] = time[1].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
        ingredients = soup.find_all('span', class_='ingredients-item-name')
        ingredient_list = [ingredient.text.replace('\n',"").strip() for ingredient in ingredients]
        nutrients = soup.find_all('span', class_='nutrient-name')
        nutrient_dict = {}
        for nutrient in nutrients:
            name = nutrient.text.replace('\n',"").strip().split(':')
            nutrient_dict[name[0]] = name[1].strip()
        image = soup.find('div', class_='image-container').noscript.img['src']
        
        print('---------------------')
        print(k)
        print(title)
        print(rating)
        print(num_ratings)
        print(time_dict)
        print(ingredient_list)
        print(nutrient_dict)
        print(collection)
        
        post = {
            'title': title,
            'rating': rating,
            'number_of_ratings': num_ratings,
            'times': time_dict,
            'ingredients': ingredient_list,
            'nutrients': nutrient_dict,
            'image_url': image
        }
        
        collection.insert_one(post)

https://www.allrecipes.com/recipe/223467/granola-bars/
---------------------
Healthy Snack Recipes
Granola Bars
Rating: 4.35 stars
107 Ratings
{'prep': '15 mins', 'total': '25 mins'}
['2 cups rolled oats', '½ cup shredded coconut', '½ cup honey', '2 tablespoons creamy peanut butter', '1 teaspoon vanilla extract', '⅛ teaspoon salt']
{'protein': '3.9g', 'carbohydrates': '34.4g', 'dietary fiber': '2.8g', 'sugars': '19.7g', 'fat': '4.7g', 'saturated fat': '1.9g', 'niacin equivalents': '1.6mg', 'vitamin c': '0.1mg', 'folate': '10.1mcg', 'calcium': '14.1mg', 'iron': '1.1mg', 'magnesium': '37.1mg', 'potassium': '128.2mg', 'sodium': '70.3mg', 'thiamin': '0.2mg', 'calories from fat': '42'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Healthy Snack Recipes')
https://www.allrecipes.com/recipe/15006/spiced-sweet-roasted-red-pepper-hummus/
---------------------
Healthy Snack Recipes
Spiced Sweet Roasted Red Pepper Hummu

---------------------
Healthy Snack Recipes
Apple Frog for Kids
Rating: 5 stars
3 Ratings
{'prep': '5 mins', 'total': '5 mins'}
['¼  Granny Smith apple, cored', '3  green grapes, halved', '1 teaspoon cream cheese, softened', '2  raisins']
{'protein': '0.6g', 'carbohydrates': '7.9g', 'dietary fiber': '0.8g', 'sugars': '6.7g', 'fat': '1.8g', 'saturated fat': '1.1g', 'cholesterol': '5.3mg', 'vitamin a iu': '79.4IU', 'niacin equivalents': '0.2mg', 'vitamin c': '3.4mg', 'folate': '1.3mcg', 'calcium': '7.7mg', 'iron': '0.2mg', 'magnesium': '2.9mg', 'potassium': '78.9mg', 'sodium': '15mg', 'calories from fat': '16'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Healthy Snack Recipes')
https://www.allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/
---------------------
Healthy Snack Recipes
Annie's Fruit Salsa and Cinnamon Chips
Rating: 4.78 stars
4,527 Ratings
{'prep': '15 mins', 'total': '10 mins'}

---------------------
Healthy Snack Recipes
Baked Kale Chips
Rating: 4.48 stars
2,597 Ratings
{'prep': '10 mins', 'total': '10 mins'}
['1 bunch kale', '1 tablespoon olive oil', '1 teaspoon seasoned salt']
{'protein': '2.5g', 'carbohydrates': '7.6g', 'dietary fiber': '1.5g', 'fat': '2.8g', 'saturated fat': '0.4g', 'vitamin a iu': '11521.1IU', 'niacin equivalents': '1.3mg', 'vitamin b6': '0.2mg', 'vitamin c': '89.7mg', 'folate': '21.7mcg', 'calcium': '104mg', 'iron': '1.3mg', 'magnesium': '25.7mg', 'potassium': '337.2mg', 'sodium': '185.1mg', 'thiamin': '0.1mg', 'calories from fat': '25.1'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Healthy Snack Recipes')
https://www.allrecipes.com/recipe/87305/microwave-popcorn/
---------------------
Healthy Snack Recipes
Microwave Popcorn
Rating: 4.55 stars
465 Ratings
{'prep': '2 mins', 'total': '3 mins'}
['½ cup unpopped popcorn', '1 teaspoon vegetable oil', '½ teaspoo

---------------------
Healthy Snack Recipes
Playgroup Granola Bars
Rating: 4.54 stars
1,958 Ratings
{'prep': '15 mins', 'total': '35 mins'}
['2 cups rolled oats', '¾ cup packed brown sugar', '½ cup wheat germ', '¾ teaspoon ground cinnamon', '1 cup all-purpose flour', '¾ cup raisins  (Optional)', '¾ teaspoon salt', '½ cup honey', '1  egg, beaten', '½ cup vegetable oil', '2 teaspoons vanilla extract']
{'protein': '2.4g', 'carbohydrates': '26.6g', 'dietary fiber': '1.4g', 'sugars': '15.7g', 'fat': '5.5g', 'saturated fat': '0.9g', 'cholesterol': '7.8mg', 'vitamin a iu': '10.4IU', 'niacin equivalents': '1.2mg', 'vitamin b6': '0.1mg', 'vitamin c': '0.2mg', 'folate': '19.9mcg', 'calcium': '15.8mg', 'iron': '0.9mg', 'magnesium': '18.9mg', 'potassium': '106.5mg', 'sodium': '79.2mg', 'thiamin': '0.1mg', 'calories from fat': '49.5'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Healthy Snack Recipes')
https://www.allre

---------------------
Low-Calorie Recipes
Red Lentil Curry
Rating: 4.5 stars
1,422 Ratings
{'prep': '10 mins', 'total': '30 mins'}
['2 cups red lentils', '1 large onion, diced', '1 tablespoon vegetable oil', '2 tablespoons curry paste', '1 tablespoon curry powder', '1 teaspoon ground turmeric', '1 teaspoon ground cumin', '1 teaspoon chili powder', '1 teaspoon salt', '1 teaspoon white sugar', '1 teaspoon minced garlic', '1 teaspoon minced fresh ginger', '1 (14.25 ounce) can tomato puree']
{'protein': '12.1g', 'carbohydrates': '32.5g', 'dietary fiber': '11.3g', 'sugars': '6.6g', 'fat': '2.6g', 'saturated fat': '0.3g', 'vitamin a iu': '376.8IU', 'niacin equivalents': '3.9mg', 'vitamin b6': '0.3mg', 'vitamin c': '8.6mg', 'folate': '227.4mcg', 'calcium': '44.1mg', 'iron': '5.5mg', 'magnesium': '61mg', 'potassium': '726.9mg', 'sodium': '571.9mg', 'thiamin': '0.2mg', 'calories from fat': '23.1'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, con

...

---------------------
Low-Fat Recipes
Brandied Cherry Clafouti
Rating: 4.47 stars
17 Ratings
{'prep': '15 mins', 'total': '50 mins'}
['2 cups canned tart cherries, drained', '¼ cup brandy', '⅔ cup white sugar, divided', 'cooking spray (such as Pam®)', '1 cup milk', '⅔ cup sifted all-purpose flour', '3  large eggs', '1 tablespoon vanilla extract', '1 tablespoon lemon zest', '¼ teaspoon salt', '⅛ teaspoon allspice', "1 teaspoon confectioners' sugar, or as needed"]
{'protein': '4.9g', 'carbohydrates': '32.4g', 'dietary fiber': '1.1g', 'sugars': '23.4g', 'fat': '2.7g', 'saturated fat': '1g', 'cholesterol': '72.2mg', 'vitamin a iu': '609.5IU', 'niacin equivalents': '1.7mg', 'vitamin b6': '0.1mg', 'vitamin c': '2.3mg', 'folate': '34.4mcg', 'calcium': '56.1mg', 'iron': '1.7mg', 'magnesium': '11.9mg', 'potassium': '147.1mg', 'sodium': '116.2mg', 'thiamin': '0.1mg', 'calories from fat': '24'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=

---------------------
Weight-Loss Recipes
Baked Beans from Scratch
Rating: 4.5 stars
122 Ratings
{'prep': '10 mins', 'total': '8 hrs'}
['1 cup dried navy beans', '4 cups water', '¼ cup ketchup', '¼ cup maple syrup', '2 tablespoons brown sugar', '2 tablespoons molasses', '1 teaspoon Worcestershire sauce', '½ teaspoon salt', '⅛ teaspoon ground black pepper', '⅛ teaspoon chili powder', '1 small onion, chopped']
{'protein': '4.8g', 'carbohydrates': '25.9g', 'dietary fiber': '5.2g', 'sugars': '12.1g', 'fat': '0.4g', 'vitamin a iu': '66.5IU', 'niacin equivalents': '1.5mg', 'vitamin b6': '0.1mg', 'vitamin c': '2.1mg', 'folate': '77.8mcg', 'calcium': '52.7mg', 'iron': '1.5mg', 'magnesium': '50.4mg', 'potassium': '364.3mg', 'sodium': '196.1mg', 'thiamin': '0.1mg', 'calories from fat': '3.3'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Weight-Loss Recipes')
https://www.allrecipes.com/recipe/9171/pattys-mashed-turnip

---------------------
Weight-Loss Recipes
Blackened Chicken
Rating: 4.38 stars
1,181 Ratings
{'prep': '10 mins', 'total': '10 mins'}
['½ teaspoon paprika', '⅛ teaspoon salt', '¼ teaspoon cayenne pepper', '¼ teaspoon ground cumin', '¼ teaspoon dried thyme', '⅛ teaspoon ground white pepper', '⅛ teaspoon onion powder', '2  skinless, boneless chicken breast halves']
{'protein': '24.7g', 'carbohydrates': '0.9g', 'dietary fiber': '0.4g', 'sugars': '0.1g', 'fat': '3g', 'saturated fat': '0.8g', 'cholesterol': '67.2mg', 'vitamin a iu': '421.1IU', 'niacin equivalents': '15.7mg', 'vitamin b6': '0.5mg', 'vitamin c': '0.7mg', 'folate': '4.8mcg', 'calcium': '20mg', 'iron': '1.4mg', 'magnesium': '26mg', 'potassium': '227.7mg', 'sodium': '204.7mg', 'thiamin': '0.1mg', 'calories from fat': '27.1'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Weight-Loss Recipes')
https://www.allrecipes.com/recipe/24712/ginger-veggie-stir-fr

---------------------
Weight-Loss Recipes
Best Ever Split Pea
Rating: 4.64 stars
261 Ratings
{'prep': '20 mins', 'total': '2 hrs 30 mins'}
['1 tablespoon olive oil', '2 cups chopped onion', '2 cups chopped carrot', '2 cups finely chopped celery', '½ tablespoon minced garlic', '1 cup yellow split peas', '1 cup green split peas', '8 cups fat-free chicken broth', '1\u2009½ teaspoons salt-free seasoning blend', '1 teaspoon salt']
{'protein': '11.2g', 'carbohydrates': '18.4g', 'dietary fiber': '6.1g', 'sugars': '5g', 'fat': '2.1g', 'saturated fat': '0.3g', 'vitamin a iu': '5580IU', 'niacin equivalents': '1.8mg', 'vitamin b6': '0.2mg', 'vitamin c': '5.1mg', 'folate': '48.4mcg', 'calcium': '36.6mg', 'iron': '1mg', 'magnesium': '28.7mg', 'potassium': '400.9mg', 'sodium': '723.4mg', 'thiamin': '0.1mg', 'calories from fat': '18.5'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Weight-Loss Recipes')
https://www.allreci

---------------------
Weight-Loss Recipes
Deer Jerky
Rating: 4.73 stars
128 Ratings
{'prep': '30 mins', 'total': '8 hrs'}
['1 pound boneless venison roast', '4 tablespoons soy sauce', '4 tablespoons Worcestershire sauce', '2 tablespoons liquid smoke flavoring', '1 tablespoon ketchup', '¼ teaspoon pepper', '¼ teaspoon garlic powder', '¼ teaspoon onion salt', '½ teaspoon salt']
{'protein': '13.6g', 'carbohydrates': '2.9g', 'dietary fiber': '0.1g', 'sugars': '1.4g', 'fat': '4.8g', 'saturated fat': '1g', 'cholesterol': '48.2mg', 'vitamin a iu': '24.4IU', 'niacin equivalents': '4mg', 'vitamin b6': '0.2mg', 'vitamin c': '1.4mg', 'folate': '4.3mcg', 'calcium': '14.4mg', 'iron': '2.6mg', 'magnesium': '18.2mg', 'potassium': '274.9mg', 'sodium': '786.1mg', 'thiamin': '0.1mg', 'calories from fat': '42.8'}
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipe_db'), 'Weight-Loss Recipes')
https://www.allrecipes.com/recipe/222247/vegan-